---
## 1. 필요한 라이브러리들을 불러온다.

In [14]:
##1. selenium, time, pandas 모듈을 불러온다.
from selenium import webdriver # selenium webdriver 개체 필수 요소.
from selenium.webdriver.common.by import By # 태그를 가져올 때 사용함.
from selenium.webdriver.support.ui import WebDriverWait # 특정 태그가 로드 될 때까지 기다리게 할 수 있음.
from selenium.webdriver.support import expected_conditions as EC # 특정 태그가 로드 될 때까지 기다리게 할 수 있음.
from selenium.webdriver.common.keys import Keys                         
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup                                                # 셀레니움 가져온 html 파일을 BeautifulSoup로 파싱하고자 할 때.

#import urllib.request                                                        # 이미지를 저장하기 위해 필요한 모듈.
#import pickle                                                                # 객체를 저장.

import time
import pandas as pd
import random
import pickle

# 이미지를 불러오지 않아 로딩이 빠르게 되게 하는 코드.
chrome_options = Options()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")

---
## 2. 구글 번역 사이트에서 직접 값을 입력하고 번역 결과를 받아서 리스트에 저장한다.

In [2]:
# dt = pd.read_csv("raw_sentic5.csv")
concepts = pd.read_csv("raw_sentic5.csv")["Unnamed: 0"].tolist()
sensitivity = pd.read_csv("raw_sentic5.csv")['sensitivity'].tolist()

In [3]:
## concept이 '_(underbar)'로 연결 된 경우가 있으면, 언더바를 지우고 띄어쓰기로 대체한다.
def remove_(x):
    if type(x) == float:
        result = x
    elif x.find("_") != -1:
        result = " ".join(x.split("_"))
    else:
        result = x
    return result

ppc_concepts = list(map(remove_, concepts))

In [5]:
driver = webdriver.Chrome("./chromedriver", options=chrome_options)

driver.implicitly_wait(5) # 페이지가 로드 된 후 바로 태그를 찾을 때 html 페이지가 완전히 로드 되지 않아 에러가 발생할 때가 있다. 이때 에러가 발생해서 스크립트 실행이 멈추는 것을 방지하기 위해 암묵적으로 5초동안 기다렸다가 스크립트를 실행하게 해준다.

driver.get("https://translate.google.co.kr/")

results = {}

debug_idx = []

start_time = time.time()


## 크롤링을 실시한다.
i = 0
for ele in ppc_concepts:
    try:
        inp = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "textarea#source"))) # 영어 concept를 입력하는 창이 DOM에 올라왔는지 확인하고, 최대 10초간 기다린다.
        inp.send_keys(ele)
        
        time.sleep(1)
        
        res = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span#result_box > span")))
        results[ele] = res.text
        
        inp.clear()
        
    except:
        print("페이지를 정상적으로 로드하지 못했습니다.")
        
        debug_idx.append(i)
        
    finally:
        if (i+1) % 50 == 0: print("{}번째 단어('{}')까지 완료 되었습니다. 소요시간:{}hours".format(i+1, 
                                                                                  ppc_concepts[i], 
                                                                                  (time.time()-start_time)/3600))
        i += 1
        

## 크롤링이 안 된 단어들만 모아서 다시 크롤링을 실시한다.
re_searchs = [ppc_concepts[i] for i in debug_idx]
re_debug_idx = []

i = 0
for ele in re_searchs:
    try:
        inp = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "textarea#source"))) # 영어 concept를 입력하는 창이 DOM에 올라왔는지 확인하고, 최대 10초간 기다린다.
        inp.send_keys(ele)
        
        time.sleep(2)
        
        res = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span#result_box > span")))
        results[ele] = res.text
        
        inp.clear()
        
    except:
        print("페이지를 정상적으로 로드하지 못했습니다.")
        
        re_debug_idx.append(i)
        
    finally:
        if (i+1) % 50 == 0: print("{}번째 단어('{}')까지 완료 되었습니다. 소요시간:{}hours".format(i+1, 
                                                                                  re_searchs[i], 
                                                                                  (time.time()-start_time)/3600))
        i += 1

50번째 단어('abdicate')까지 완료 되었습니다. 소요시간:0.0152305934826533hours
100번째 단어('abject misery')까지 완료 되었습니다. 소요시간:0.030478419065475465hours
150번째 단어('abominable')까지 완료 되었습니다. 소요시간:0.045715744561619226hours
200번째 단어('abscission')까지 완료 되었습니다. 소요시간:0.060880578358968096hours
250번째 단어('absolutely impossible')까지 완료 되었습니다. 소요시간:0.0761469480726454hours
300번째 단어('abstruseness')까지 완료 되었습니다. 소요시간:0.09143420808845096hours
350번째 단어('academic advisor')까지 완료 되었습니다. 소요시간:0.1066750830411911hours
400번째 단어('access')까지 완료 되었습니다. 소요시간:0.12192041714986165hours
450번째 단어('acclimatizing')까지 완료 되었습니다. 소요시간:0.13721503098805746hours
500번째 단어('accrue')까지 완료 되었습니다. 소요시간:0.15246159699228073hours
550번째 단어('acer saccharum')까지 완료 되었습니다. 소요시간:0.16767655067973666hours
600번째 단어('aching feet')까지 완료 되었습니다. 소요시간:0.18289170185724893hours
650번째 단어('acknowledged')까지 완료 되었습니다. 소요시간:0.19808032949765522hours
700번째 단어('acral')까지 완료 되었습니다. 소요시간:0.21336121949884626hours
750번째 단어('act unilaterally')까지 완료 되었습니다. 소요시간:0.22899633857938978hours
800

6200번째 단어('balzan prize')까지 완료 되었습니다. 소요시간:1.913275865316391hours
6250번째 단어('banger')까지 완료 되었습니다. 소요시간:1.9286998855405384hours
6300번째 단어('banning junk food')까지 완료 되었습니다. 소요시간:1.9443859800365235hours
6350번째 단어('barbecue dinner')까지 완료 되었습니다. 소요시간:1.9599633879794014hours
6400번째 단어('barely comprehensible')까지 완료 되었습니다. 소요시간:1.9755620969004102hours
6450번째 단어('barnard star')까지 완료 되었습니다. 소요시간:1.9911868955691656hours
6500번째 단어('barren desert')까지 완료 되었습니다. 소요시간:2.0068125331401827hours
6550번째 단어('base stealer')까지 완료 되었습니다. 소요시간:2.022443103790283hours
6600번째 단어('basidiomycete')까지 완료 되었습니다. 소요시간:2.038019171688292hours
6650번째 단어('bass viol')까지 완료 되었습니다. 소요시간:2.0535555356078676hours
6700번째 단어('bathing cap')까지 완료 되었습니다. 소요시간:2.0690491525994408hours
6750번째 단어('battersea power station')까지 완료 되었습니다. 소요시간:2.084647387133704hours
6800번째 단어('bayer designation')까지 완료 되었습니다. 소요시간:2.1002075524462596hours
6850번째 단어('beach soccer league')까지 완료 되었습니다. 소요시간:2.1158455638753044hours
6900번째 단어('bear upon')까지 완료 되었습니다.

12350번째 단어('card type')까지 완료 되었습니다. 소요시간:3.8496678495407104hours
12400번째 단어('cardiovascular pathology')까지 완료 되었습니다. 소요시간:3.8659118447701135hours
12450번째 단어('cargo compartment')까지 완료 되었습니다. 소요시간:3.8819345196750428hours
12500번째 단어('carnauba palm')까지 완료 되었습니다. 소요시간:3.897886595858468hours
12550번째 단어('carriage house')까지 완료 되었습니다. 소요시간:3.9140403251515496hours
12600번째 단어('carry weight')까지 완료 되었습니다. 소요시간:3.9299544827805626hours
12650번째 단어('cash')까지 완료 되었습니다. 소요시간:3.9459573039743634hours
12700번째 단어('cassia bark')까지 완료 되었습니다. 소요시간:3.9618250991900763hours
12750번째 단어('casualty evacuation')까지 완료 되었습니다. 소요시간:3.9776903673675323hours
12800번째 단어('catastrophic')까지 완료 되었습니다. 소요시간:3.9937046498060225hours
12850번째 단어('catchment basin')까지 완료 되었습니다. 소요시간:4.009659601781103hours
12900번째 단어('catholic')까지 완료 되었습니다. 소요시간:4.025511977937486hours
12950번째 단어('catwalk')까지 완료 되었습니다. 소요시간:4.041586931149165hours
13000번째 단어('cause diarrhoea')까지 완료 되었습니다. 소요시간:4.057554929455121hours
13050번째 단어('cautionary note')까지 완료 되었습니다.

18400번째 단어('conventional therapy')까지 완료 되었습니다. 소요시간:5.803489715721872hours
페이지를 정상적으로 로드하지 못했습니다.
18450번째 단어('conveyor')까지 완료 되었습니다. 소요시간:5.81982844306363hours
18500번째 단어('cook stove')까지 완료 되었습니다. 소요시간:5.835829060408805hours
18550번째 단어('cool whip')까지 완료 되었습니다. 소요시간:5.851765334142579hours
18600번째 단어('copied onto')까지 완료 되었습니다. 소요시간:5.867938625746303hours
18650번째 단어('coral')까지 완료 되었습니다. 소요시간:5.883895938396454hours
18700번째 단어('cormorant')까지 완료 되었습니다. 소요시간:5.899887906842761hours
18750번째 단어('cornmeal')까지 완료 되었습니다. 소요시간:5.915820428133011hours
18800번째 단어('corpse')까지 완료 되었습니다. 소요시간:5.93198066174984hours
18850번째 단어('correspondent')까지 완료 되었습니다. 소요시간:5.9483602813217376hours
18900번째 단어('corticoid')까지 완료 되었습니다. 소요시간:5.964848398698701hours
18950번째 단어('cost-efficient')까지 완료 되었습니다. 소요시간:5.981023306846619hours
19000번째 단어('cotton boll')까지 완료 되었습니다. 소요시간:5.996981812914212hours
19050번째 단어('count indictment')까지 완료 되었습니다. 소요시간:6.013181136846542hours
19100번째 단어('counterinsurgency mission')까지 완료 되었습니다. 소요시간:6.

24350번째 단어('distinctively human')까지 완료 되었습니다. 소요시간:7.728750988311238hours
24400번째 단어('distributed system compiler')까지 완료 되었습니다. 소요시간:7.745599235759841hours
24450번째 단어('diva')까지 완료 되었습니다. 소요시간:7.7621082311868665hours
24500번째 단어('divide between')까지 완료 되었습니다. 소요시간:7.77873889863491hours
24550번째 단어('diving duck')까지 완료 되었습니다. 소요시간:7.795530139274067hours
24600번째 단어('do drug')까지 완료 되었습니다. 소요시간:7.812077397637897hours
24650번째 단어('document')까지 완료 되었습니다. 소요시간:7.828480007251104hours
24700번째 단어('dog chase')까지 완료 되었습니다. 소요시간:7.845045578744676hours
24750번째 단어('doing brisk')까지 완료 되었습니다. 소요시간:7.861265491776996hours
24800번째 단어('domed mosque')까지 완료 되었습니다. 소요시간:7.8777628825108215hours
24850번째 단어('dominant ideology')까지 완료 되었습니다. 소요시간:7.894540300369263hours
24900번째 단어('donor kidney')까지 완료 되었습니다. 소요시간:7.911003957457012hours
24950번째 단어('dopamine agonist')까지 완료 되었습니다. 소요시간:7.92708623263571hours
25000번째 단어('doting')까지 완료 되었습니다. 소요시간:7.94347194592158hours
25050번째 단어('double talk')까지 완료 되었습니다. 소요시간:7.9604793651236

...

76950번째 단어('sap')까지 완료 되었습니다. 소요시간:26.575949500931635hours
77000번째 단어('sassing')까지 완료 되었습니다. 소요시간:26.593334877093632hours
77050번째 단어('satirist lucian')까지 완료 되었습니다. 소요시간:26.613822349243694hours
77100번째 단어('satyrinae subfamily')까지 완료 되었습니다. 소요시간:26.63257966823048hours
77150번째 단어('savitri devi')까지 완료 되었습니다. 소요시간:26.652231955197124hours
77200번째 단어('say welcome')까지 완료 되었습니다. 소요시간:26.669696942302917hours
77250번째 단어('scalpel')까지 완료 되었습니다. 소요시간:26.68960318532255hours
77300번째 단어('scarab')까지 완료 되었습니다. 소요시간:26.70763894577821hours
페이지를 정상적으로 로드하지 못했습니다.
77350번째 단어('scatter diagram')까지 완료 되었습니다. 소요시간:26.727463081214164hours
77400번째 단어('scheduled passenger train')까지 완료 되었습니다. 소요시간:26.745899358259308hours
77450번째 단어('schnapps')까지 완료 되었습니다. 소요시간:26.766474530299504hours
77500번째 단어('schoolbag')까지 완료 되었습니다. 소요시간:26.7852967170212hours
77550번째 단어('science laboratory')까지 완료 되었습니다. 소요시간:26.805360623995462hours
77600번째 단어('scoffer')까지 완료 되었습니다. 소요시간:26.82428865664535hours
77650번째 단어('scotoma')까지 완료 되었습니다. 소요시

82750번째 단어('solicitousness')까지 완료 되었습니다. 소요시간:28.807521365152464hours
82800번째 단어('solo ascent')까지 완료 되었습니다. 소요시간:28.82616811533769hours
82850번째 단어('somatosensory cortex')까지 완료 되었습니다. 소요시간:28.846928525898193hours
82900번째 단어('songful')까지 완료 되었습니다. 소요시간:28.864896758264965hours
82950번째 단어('sophrosyne')까지 완료 되었습니다. 소요시간:28.885439694391355hours
83000번째 단어('sort clothes')까지 완료 되었습니다. 소요시간:28.90315507332484hours
83050번째 단어('sound power')까지 완료 되었습니다. 소요시간:28.921397782431708hours
83100번째 단어('soutane')까지 완료 되었습니다. 소요시간:28.941108091804715hours
83150번째 단어('southward')까지 완료 되었습니다. 소요시간:28.960510696834987hours
페이지를 정상적으로 로드하지 못했습니다.
83200번째 단어('soya milk')까지 완료 되었습니다. 소요시간:28.981575096249582hours
83250번째 단어('span')까지 완료 되었습니다. 소요시간:29.000385593970616hours
83300번째 단어('sparkle')까지 완료 되었습니다. 소요시간:29.018954633673033hours
83350번째 단어('spay')까지 완료 되었습니다. 소요시간:29.039797089563475hours
83400번째 단어('special collection department')까지 완료 되었습니다. 소요시간:29.058582363327343hours
83450번째 단어('specimen')까지 완료 되었습니다. 소요시간:2

88500번째 단어('team up')까지 완료 되었습니다. 소요시간:31.040134596758417hours
88550번째 단어('technical advisory')까지 완료 되었습니다. 소요시간:31.058504726224477hours
88600번째 단어('technology organisation dsto')까지 완료 되었습니다. 소요시간:31.078841418094104hours
페이지를 정상적으로 로드하지 못했습니다.
88650번째 단어('teeth whitening')까지 완료 되었습니다. 소요시간:31.099248242908054hours
88700번째 단어('telepathic')까지 완료 되었습니다. 소요시간:31.118056817584566hours
88750번째 단어('television broadcaster')까지 완료 되었습니다. 소요시간:31.139624962740474hours
88800번째 단어('temerity')까지 완료 되었습니다. 소요시간:31.157924690710175hours
88850번째 단어('temporarily stepped down')까지 완료 되었습니다. 소요시간:31.176923292279245hours
페이지를 정상적으로 로드하지 못했습니다.
88900번째 단어('tendency')까지 완료 되었습니다. 소요시간:31.198714554574753hours
88950번째 단어('tenor solo')까지 완료 되었습니다. 소요시간:31.216728981667096hours
89000번째 단어('tepal white')까지 완료 되었습니다. 소요시간:31.23766558362378hours
페이지를 정상적으로 로드하지 못했습니다.
89050번째 단어('terrestrial')까지 완료 되었습니다. 소요시간:31.256073128912185hours
페이지를 정상적으로 로드하지 못했습니다.
89100번째 단어('terrorist group')까지 완료 되었습니다. 소요시간:31.277538482281898

페이지를 정상적으로 로드하지 못했습니다.
93750번째 단어('unmaintainable')까지 완료 되었습니다. 소요시간:33.146109977033404hours
93800번째 단어('unnecessarily cruel')까지 완료 되었습니다. 소요시간:33.16840794490443hours
페이지를 정상적으로 로드하지 못했습니다.
93850번째 단어('unpersuasiveness')까지 완료 되었습니다. 소요시간:33.188876764310734hours
페이지를 정상적으로 로드하지 못했습니다.
93900번째 단어('unprotected oral sex')까지 완료 되었습니다. 소요시간:33.2118852783574hours
93950번째 단어('unregistered vehicle')까지 완료 되었습니다. 소요시간:33.2319682827923hours
94000번째 단어('unsated')까지 완료 되었습니다. 소요시간:33.253871728248065hours
페이지를 정상적으로 로드하지 못했습니다.
94050번째 단어('unsettled')까지 완료 되었습니다. 소요시간:33.27393697546588hours
페이지를 정상적으로 로드하지 못했습니다.
94100번째 단어('unspecific')까지 완료 되었습니다. 소요시간:33.29368503411611hours
94150번째 단어('unsusceptibility')까지 완료 되었습니다. 소요시간:33.31299228025807hours
페이지를 정상적으로 로드하지 못했습니다.
94200번째 단어('untouchability')까지 완료 되었습니다. 소요시간:33.334443106386395hours
페이지를 정상적으로 로드하지 못했습니다.
94250번째 단어('unveiling ceremony')까지 완료 되었습니다. 소요시간:33.354176231622695hours
94300번째 단어('unworldly')까지 완료 되었습니다. 소요시간:33.3763337616788hours
페이지를 

98900번째 단어('wooden box')까지 완료 되었습니다. 소요시간:35.27483135335975hours
페이지를 정상적으로 로드하지 못했습니다.
98950번째 단어('wooden table')까지 완료 되었습니다. 소요시간:35.29525370909108hours
페이지를 정상적으로 로드하지 못했습니다.
99000번째 단어('woolgather')까지 완료 되었습니다. 소요시간:35.314813580314315hours
99050번째 단어('work in garden')까지 완료 되었습니다. 소요시간:35.33436985658275hours
페이지를 정상적으로 로드하지 못했습니다.
99100번째 단어('working undercover')까지 완료 되었습니다. 소요시간:35.35654481616285hours
99150번째 단어('world largest retailer')까지 완료 되었습니다. 소요시간:35.37697310109933hours
99200번째 단어('worrywart')까지 완료 되었습니다. 소요시간:35.39722243183189hours
99250번째 단어('woven into')까지 완료 되었습니다. 소요시간:35.41984683725569hours
99300번째 단어('wrecked car')까지 완료 되었습니다. 소요시간:35.439077221949894hours
페이지를 정상적으로 로드하지 못했습니다.
페이지를 정상적으로 로드하지 못했습니다.
99350번째 단어('write down')까지 완료 되었습니다. 소요시간:35.46082446157932hours
99400번째 단어('written message')까지 완료 되었습니다. 소요시간:35.48079657415549hours
99450번째 단어('wych elm ulmus glabra')까지 완료 되었습니다. 소요시간:35.49986825591988hours
페이지를 정상적으로 로드하지 못했습니다.
페이지를 정상적으로 로드하지 못했습니다.
99500번째 단어('xy 

In [38]:
print(len(results))
print(results['weather'])

99725
날씨


In [39]:
with open('sentic_to_korean_dic.pkl', 'wb') as savedata:
    pickle.dump(obj=results, file=savedata)

----
## 3. 데이터프레임 형태로 만들고 csv파일로 저장한다.

In [104]:
## Full dictionary
sentic2korean_full = pd.DataFrame({'sentic_word': list(results.keys()), 'korean': list(results.values())})
sentic2korean_full.to_csv("sentic2korean_full.csv")

In [133]:
## Unigram dictionary
idx = (sentic2korean.sentic_word.str.find(" ") == -1)

sentic2korean_unigram = sentic2korean[idx]
sentic2korean_unigram.index = range(sentic2korean_unigram.shape[0])

sentic2korean_unigram.head(5)

sentic2korean_unigram.to_csv("sentic2korean_unigram.csv")